In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.empty_cache()

In [4]:
# preprocess
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, out.size()[2:])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def Net():
    return ResNet(Bottleneck, [3, 4, 6, 3])

In [7]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
# different optimizers
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.003, total_steps=100 * len(trainloader), pct_start=0.3)

In [8]:
def train_model(num_epochs=100, patience=10):
    model.train()
    best_acc = 0.0
    patience_counter = 0
    steps_per_epoch = len(trainloader)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            scheduler.step()
            
            running_loss += loss.item()
            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss/100:.4f}')
                running_loss = 0.0

        # Early stopping
        acc = test_model()
        if acc > best_acc + 0.001:
            path = './models/resnet_optim.pth'
            torch.save(model.state_dict(), path)
            patience_counter = 0
        else:
            patience_counter += 1
            print(f'Patience counter: {patience_counter}/{patience}')
            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch+1}')
                break


In [11]:
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f'Accuracy on test set: {acc:.2f}%')
    return acc
    

In [12]:
train_model(num_epochs=100)

Epoch [1/100], Step [100], Loss: 1.3831
Epoch [1/100], Step [200], Loss: 1.3197
Epoch [1/100], Step [300], Loss: 1.2370
Accuracy on test set: 58.57%
Epoch [2/100], Step [100], Loss: 1.5204
Epoch [2/100], Step [200], Loss: 1.3092
Epoch [2/100], Step [300], Loss: 1.2373
Accuracy on test set: 61.74%
Epoch [3/100], Step [100], Loss: 1.1383
Epoch [3/100], Step [200], Loss: 1.1827
Epoch [3/100], Step [300], Loss: 1.0510
Accuracy on test set: 65.34%
Epoch [4/100], Step [100], Loss: 0.9929
Epoch [4/100], Step [200], Loss: 0.9972
Epoch [4/100], Step [300], Loss: 0.9959
Accuracy on test set: 67.71%
Epoch [5/100], Step [100], Loss: 0.9420
Epoch [5/100], Step [200], Loss: 0.9149
Epoch [5/100], Step [300], Loss: 0.9154
Accuracy on test set: 71.45%
Epoch [6/100], Step [100], Loss: 0.8711
Epoch [6/100], Step [200], Loss: 0.8358
Epoch [6/100], Step [300], Loss: 0.8423
Accuracy on test set: 70.65%
Epoch [7/100], Step [100], Loss: 0.8092
Epoch [7/100], Step [200], Loss: 0.7606
Epoch [7/100], Step [300],

ValueError: Tried to step 39101 times. The specified number of total steps is 39100

In [13]:
test_model()

Accuracy on test set: 92.14%


92.14

In [14]:
# save model
path = './models/resnet_optim.pth'
torch.save(model.state_dict(), path)